In [ ]:


import numpy as np
import pandas as pd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/madhavgloves/glove.twitter.27B.25d.pkl
/kaggle/input/madhavtweeterdata/training.1600000.processed.noemoticon.csv


In [ ]:
import pandas as pd


file_path = '/kaggle/input/madhavtweeterdata/training.1600000.processed.noemoticon.csv'


columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
data = pd.read_csv(file_path, encoding='ISO-8859-1', names=columns)


data = data[['sentiment', 'text']]


data['sentiment'] = data['sentiment'].map({0: 'negative', 4: 'positive'})


In [ ]:
import pickle


glove_path = '/kaggle/input/madhavgloves/glove.twitter.27B.25d.pkl'

with open(glove_path, 'rb') as f:
    glove_embeddings = pickle.load(f)


In [ ]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(data, test_size=0.2, random_state=42)
train, val = train_test_split(train_val, test_size=0.25, random_state=42)


In [ ]:
import numpy as np

def preprocess_text(text):
    text = text.lower()
    return text

def text_to_embedding(text, embeddings):
    tokens = text.split()
    embeddings_list = [embeddings.get(token, np.zeros(25)) for token in tokens]

    embedding = np.mean(embeddings_list, axis=0) if embeddings_list else np.zeros(25)
    return embedding


train['text'] = train['text'].apply(preprocess_text)
val['text'] = val['text'].apply(preprocess_text)
test['text'] = test['text'].apply(preprocess_text)


X_train = np.array([text_to_embedding(text, glove_embeddings) for text in train['text']])
X_val = np.array([text_to_embedding(text, glove_embeddings) for text in val['text']])
X_test = np.array([text_to_embedding(text, glove_embeddings) for text in test['text']])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train['sentiment'].values)
y_val = label_encoder.transform(val['sentiment'].values)
y_test = label_encoder.transform(test['sentiment'].values)

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate on the test set
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Test accuracy: {accuracy}")


Test accuracy: 0.67374375
